# XML einlesen

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import os as os

def read_xml_file(file_path):
  """Liest eine XML-Datei ein und gibt den Wurzelknoten zurück."""
  try:
    print("file_path: ", file_path)
    tree = ET.parse(file_path)
    root = tree.getroot()
    return root
  except FileNotFoundError:
    print(f"Datei nicht gefunden: {file_path}")
    return None
  except ET.ParseError:
    print(f"Fehler beim Parsen der XML-Datei: {file_path}")
    return None


In [ ]:
# Beispiel: XML-Datei aus dem aktuellen Verzeichnis einlesen
# Ersetzen Sie 'your_xml_file.xml' durch den tatsächlichen Dateinamen
xml_file_name = '9151-21028044-000_.xml'

current_directory = os.getcwd()
xml_file_path = os.path.join(current_directory, xml_file_name)

root = read_xml_file(xml_file_path)

if root is not None:
  # Hier können Sie mit den XML-Daten arbeiten
  print("XML-Datei erfolgreich eingelesen!")
  # Beispiel: Alle Elemente im XML-Baum ausgeben
  for child in root:
    print(child.tag, child.attrib)


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Read XML File

In [ ]:
# 9151-20201689-000_.xml ohne KLB_DATEN
# 9151-00000501-000_.xml mit KLB_DATEN und Makrokontur
# 9151-00000502-000_.xml Makrokontur 4Nutes
# 9151-21028044-000_.xml Makrokontur 5Nutes
# 9151-20204659-000_.xml bis groove4 / !
# 9151-21001084-000_.xml bis groove4 / ok
# 9151-21001708-000_.xml bis groove3 / !
# 9151-21014753-000_.xml bis groove4 / !
# 9151-21018270-000_.xml bis groove4 / !
# 9151-21022736-000_.xml bis groove3 / ok
# 9151-01000972-000_.xml ohne Makro-Contur
# 9151-21043089-000_.xml bis groove3 / ok
# 9151-20183875-000_.xml bis groove4 / ok / 6DN
# 9151-20195098-000_.xml bis groove3 / ! zwei FKAs

print(uploaded)
print(fn)
root = read_xml_file(fn)



# Read the Macrocontur

In [ ]:
def readMacrocontur(root):
  klb_data_found = False
  yklb_values = []
  xklb_values = []
  if root:
    for element in root.findall(".//MAKROKONTUR"):
      for child in element:
        if child.tag == 'KLB_DATEN':
          for child2 in child:
            if child2.tag == 'ROW':
              yklb_found = False
              xklb_found = False
              for child3 in child2:
                if child3.tag == 'YKLB':
                  yklb_values.append(float(child3.text))
                  yklb_found = True
                elif child3.tag == 'XKLB':
                  xklb_values.append(float(child3.text))
                  xklb_found = True
              if not yklb_found:
                yklb_values.append(np.nan)  # Füge NaN hinzu, falls YKLB fehlt
              if not xklb_found:
                xklb_values.append(np.nan)  # Füge NaN hinzu, falls XKLB fehlt
  yklb_array = np.array(yklb_values)
  xklb_array = np.array(xklb_values)
  return xklb_array, yklb_array



In [ ]:
def readProfileLimits(root):
  level_values = []
  level_number_values = []
  if root:
    #print("Root")
    for element in root.findall(".//FKA_TAB"):
      #print("FKA_TAB")
      for child in element:
        #print(child.tag )
        for child2 in child:
          #print(child2.tag )
          if child2.tag == 'HOEHE':
              level_values.append(float(child2.text))
          if child2.tag == 'ABSCHNITT':
            #print(child2.tag )
            for child3 in child2:
              #print(child3.tag )
              if child3.tag == 'NUMMER':
                level_number_values.append(float(child3.text))
  return np.array(level_values), np.array(level_number_values)

level_limit, level_limit_number = readProfileLimits(root)
print(level_limit)
print(level_limit_number)



In [ ]:
def readGeometrieProfileCurves(root):
  xa  = []
  ya  = []
  xe  = []
  ye  = []
  level_number_values = []
  if root:
    print("Root")
    isTS  = False
    for element in root.findall(".//GEOMETRIE_FORMKURVEN"):
      #print("GEOMETRIE_FORMKURVEN")
      for child in element:
        if child.tag == 'LINE':
          for child2 in child:
            if(child2.tag == 'TYPENAME' and child2.text == 'FOK_LAUFRICHTUNG'):
              isTS = True
            if isTS == True:
              if child2.tag == 'XA':
                  #print(child2.tag, child2.text)
                  xa.append(float(child2.text))
              if child2.tag == 'YA':
                  #print(child2.tag, child2.text)
                  ya.append(float(child2.text))
              if child2.tag == 'XE':
                  #print(child2.tag, child2.text)
                  xe.append(float(child2.text))
              if child2.tag == 'YE':
                #print(child2.tag, child2.text)
                #print('------------------')
                ye.append(float(child2.text))
                isTS = False

  return np.array(xa), np.array(ya),np.array(xe), np.array(ye)

# readGeometrieProfileCurves(root)

# Correction X/Y Coordinates for Macrocontur

In [ ]:

# Reihenfolge der letzten zwei einträge im Array tauschen
def changeContur(xarray,yarray, nxklb, nyklb):
  x = xarray[nxklb-1]
  y = yarray[nyklb-1]
  xarray[nxklb-1] = xarray[nxklb]
  yarray[nyklb-1] = yarray[nyklb]
  xarray[nxklb] = x
  yarray[nxklb] = y
  return xarray, yarray


In [ ]:
  klb_data_found = True
  xklb_array, yklb_array = readMacrocontur(root)
  # i = 0
  # for x,y in zip(xklb_array,yklb_array):
  # print(i, " ",x,y)
  # i = i+1

  if xklb_array.size > 0 and yklb_array.size > 0:
    print("KLB_DATEN im XML vorhanden, tausche letzte Position")
    klb_data_found = True
    xklb_array,yklb_array = changeContur(xklb_array,yklb_array, xklb_array.size-1, yklb_array.size-1)
    i = 0
    # for x,y in zip(xklb_array,yklb_array):
    # print(i, " ",x,y)
    # i = i+1
  else:
    print("Keine KLB_DATEN im XML vorhanden")
    klb_data_found = False


In [ ]:
nom_diameter = 0.0
total_length = 0.0

top_land = 0.0
top_land_diameter = 0.0

groove1 = 0.0
groove1_diameter = 0.0

land2 = 0.0
groove2 = 0.0

land2_diameter = 0.0
groove2_diameter = 0.0

land3 = 0.0
groove3 = 0.0

land3_diameter = 0.0
groove3_diameter = 0.0

land4 = 0.0
groove4 = 0.0

land4_diameter = 0.0
groove4_diameter = 0.0

land5 = 0.0
groove5 = 0.0

land5_diameter = 0.0
groove5_diameter = 0.0

skirt_length = 0.0

if root:
  yklb_values = []
  xklb_values = []
  for element in root.findall(".//HAUPTABMESSUNGEN"):
    for child in element:
      if child.tag == 'ANZAHL_NUTEN':
        print("Anzahl Nuten: ",child.text)
      elif child.tag == 'HOEHE':
        print("Hoehe: ",child.text);
        total_length = float(child.text)

      elif child.tag == 'FEUERSTEG_LAENGE':
        print("----------------------------------------")
        print("FEUERSTEG_LAENGE = top_land_length",child.text)
        top_land = float(child.text)
      elif child.tag == 'FEUERSTEG_DURCHMESSER':
        print("FEUERSTEG_DURCHMESSER",child.text)
        top_land_diameter= float(child.text)

      elif child.tag == 'RINGSTEG4_LAENGE':
        print("----------------------------------------")
        land5 = float(child.text)
        print("RINGSTEG5_LAENGE",child.text)
      elif child.tag == 'RINGSTEG4_DURCHMESSER':
        print("RINGSTEG5_DURCHMESSER",child.text)
        land5_diameter = float(child.text)

      elif child.tag == 'RINGSTEG3_LAENGE':
        print("----------------------------------------")
        land4 = float(child.text)
        print("RINGSTEG4_LAENGE",child.text)
      elif child.tag == 'RINGSTEG3_DURCHMESSER':
        print("RINGSTEG4_DURCHMESSER",child.text)
        land4_diameter = float(child.text)

      elif child.tag == 'RINGSTEG2_LAENGE':
        print("----------------------------------------")
        land3 = float(child.text)
        print("RINGSTEG3_LAENGE",child.text)
      elif child.tag == 'RINGSTEG2_DURCHMESSER':
        print("RINGSTEG3_DURCHMESSER",child.text)
        land3_diameter = float(child.text)

      elif child.tag == 'RINGSTEG1_LAENGE':
        print("----------------------------------------")
        land2 = float(child.text)
        print("RINGSTEG2_LAENGE",child.text)
      elif child.tag == 'RINGSTEG1_DURCHMESSER':
        print("RINGSTEG2_DURCHMESSER",child.text)
        land2_diameter = float(child.text)

      elif child.tag == 'NUT5_LAENGE':
        print("----------------------------------------")
        print("NUT5_LAENGE",child.text)
        groove5 = float(child.text)
      elif child.tag == 'NUT5_DURCHMESSER':
        print("NUT5_DURCHMESSER",child.text)
        groove5_diameter = float(child.text)

      elif child.tag == 'NUT4_LAENGE':
        print("----------------------------------------")
        print("NUT4_LAENGE",child.text)
        groove4 = float(child.text)
      elif child.tag == 'NUT4_DURCHMESSER':
        print("NUT4_DURCHMESSER",child.text)
        groove4_diameter = float(child.text)

      elif child.tag == 'NUT3_LAENGE':
        print("----------------------------------------")
        print("NUT3_LAENGE",child.text)
        groove3 = float(child.text)
      elif child.tag == 'NUT3_DURCHMESSER':
        print("NUT3_DURCHMESSER",child.text)
        groove3_diameter = float(child.text)

      elif child.tag == 'NUT2_LAENGE':
        print("----------------------------------------")
        print("NUT2_LAENGE",child.text)
        groove2 = float(child.text)
      elif child.tag == 'NUT2_DURCHMESSER':
        print("NUT2_DURCHMESSER",child.text)
        groove2_diameter = float(child.text)

      elif child.tag == 'NUT1_LAENGE':
        print("----------------------------------------")
        print("NUT1_LAENGE",child.text)
        groove1 = float(child.text)
      elif child.tag == 'NUT1_DURCHMESSER':
        print("NUT1_DURCHMESSER",child.text)
        groove1_diameter = float(child.text)

      elif child.tag == 'SCHAFTDURCHMESSER':
        print("SCHAFTDURCHMESSER",child.text)
        nom_diameter = float(child.text)
      elif child.tag == 'SCHAFTLAENGE':
        print("----------------------------------------")
        print("SCHAFTLAENGE",child.text)
        skirt_length= float(child.text)
      elif child.tag == 'DURCHMESSER':
        print("DURCHMESSER",child.text)


In [ ]:
klb_data_found = False
if klb_data_found == False:
  print("keine KLB Daten vorhanden")
  nNuten = 100
  # Erstelle ein leeres Array mit der gewünschten Größe
  xklb_array = np.empty(nNuten)
  yklb_array = np.empty(nNuten)
  i = 0
  ixy = 0
  for i in range(xklb_array.size):
    xklb_array[i] = float('nan')
    yklb_array[i] = float('nan')

  # Start skirt links unten
  print("***start skirt***")
  xklb_array[ixy] = 0 #nom_diameter/2
  yklb_array[ixy] = 0
  print("1) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1

  # Start skirt hori. nach rechts unten
  xklb_array[ixy] = nom_diameter/2
  yklb_array[ixy] = 0 #skirt_length
  print("2) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1

  # Skirt vertikale rechts unten nach rechts skirt len.
  xklb_array[ixy] = nom_diameter/2
  yklb_array[ixy] = skirt_length
  print("3) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1

  # Nut
  if groove5 > 0.0:
    print("***groove5***")
    xklb_array[ixy] = groove5_diameter/2
    yklb_array[ixy] = skirt_length
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove5_diameter/2
    yklb_array[ixy] = skirt_length+groove5
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land5 > 0.0:
    print("***land5***")
    xklb_array[ixy] = land5_diameter/2
    yklb_array[ixy] = skirt_length+groove5
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land5_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if groove4 > 0.0:
    print("***groove4***")
    xklb_array[ixy] = groove4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land4 > 0.0:
    print("***land4***")
    xklb_array[ixy] = land4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if groove3 > 0.0:
    print("***groove3***")
    xklb_array[ixy] = groove3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land3 > 0.0:
    print("***land3***")
    xklb_array[ixy] = land3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1


  if groove2 > 0.0:
    print("***groove2***")
    xklb_array[ixy] = groove2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land2 > 0.0:
    print("***land2***")
    xklb_array[ixy] = land2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if groove1 > 0.0:
    print("***groove1***")
    xklb_array[ixy] = groove1_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove1_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if top_land > 0.0:
    print("***top_land (Feuersteg)***")
    xklb_array[ixy] = top_land_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    #Feuersteg
    xklb_array[ixy] = top_land_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1+top_land
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  #Ende Kolben Feuersteg horizontal
  print("Kolben Feuersteg Horizontal")
  xklb_array[ixy] = 0
  yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1+top_land
  print("1) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1
  print("********************************************")

  # Reihenfolge der xklb_array und yklb_array umdrehen
  xklb_array = xklb_array[::-1]
  yklb_array = yklb_array[::-1]

  # Entfernen von NaN-Werten aus den Arrays
  valid_indices = ~np.isnan(xklb_array) & ~np.isnan(yklb_array)
  xklb_array = xklb_array[valid_indices]
  yklb_array = yklb_array[valid_indices]

# Test Ausgabe
i = 0
for x,y in zip(xklb_array,yklb_array):
    print(i, " ",x,y)
    i = i+1
# Test Plot
# plt.plot(xklb_array, yklb_array, marker='.', linestyle='-')



# Plot Macrocontur

In [ ]:
# Plot erstellen
plt.figure(figsize=(10, 8))  # Set the figure size (width, height) in inches
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)  # sharey=True ensures the same y-axis scaling
fig.suptitle(fn)
ax1.set_title('Macrocontur')
ax2.set_title('Formkurve')

plt.xlim(0, int(nom_diameter+100))
# plt.ylim(0, int(total_length*2))
ax1.plot(xklb_array, yklb_array, marker='.', linestyle='-')

hlinewidth=1.5
hxmin=1 #nom_diameter/10
hxmax= nom_diameter/2 +1  -1#nom_diameter/2 +4
# Feuersteg Höhe
if top_land > 0:
  top_land_hight_absolute = skirt_length+groove5+groove4+groove3+groove2+groove1+land5+land4+land3+land2+top_land
  ax1.hlines(y=top_land_hight_absolute, xmin=hxmin, xmax=hxmax, colors='darkgreen', linestyles='--', linewidth=hlinewidth, label='a')
  ax1.text(hxmax+2, top_land_hight_absolute, "Top Land", color='darkgreen', fontsize=6)
  # untere kante Feuersteg
  ax1.hlines(y=top_land_hight_absolute - top_land, xmin=hxmin, xmax=hxmax, colors='darkgreen', linestyles='--', linewidth=hlinewidth, label='a')
  ax1.text(hxmax+2, top_land_hight_absolute- top_land, "Top Land", color='darkgreen', fontsize=6)

# if groove1 > 0:
#  plt.hlines(y=skirt_length+groove5+groove4+groove3+groove2+groove1+land5+land4+land3+land2, xmin=hxmin, xmax=hxmax, colors='darkgreen', linestyles='--', linewidth=hlinewidth,label='a')

if land2 > 0:
  land2_height_absolute = skirt_length+groove5+groove4+groove3+groove2+land5+land4+land3+land2
  ax1.hlines(y=land2_height_absolute, xmin=hxmin, xmax=hxmax, colors='gold', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land2_height_absolute, "2nd land", color='gold', fontsize=6)
  ax1.hlines(y=land2_height_absolute-land2, xmin=hxmin, xmax=hxmax, colors='gold', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land2_height_absolute-land2, "2nd land", color='gold', fontsize=6)

if land3 > 0:
  land3_height_absolute = skirt_length+groove5+land5+groove4+land4+groove3+land3
  ax1.hlines(y=land3_height_absolute, xmin=hxmin, xmax=hxmax, colors='purple', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land3_height_absolute, "3nd land", color='purple', fontsize=6)
  ax1.hlines(y=land3_height_absolute-land3, xmin=hxmin, xmax=hxmax, colors='purple', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land3_height_absolute-land3, "3nd land", color='purple', fontsize=6)

if land4 > 0:
  land4_height_absolute = skirt_length+groove5+land5+groove4+land4
  ax1.hlines(y=land4_height_absolute, xmin=hxmin, xmax=hxmax, colors='darkred', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land4_height_absolute, "4nd land", color='darkred', fontsize=6)
  ax1.hlines(y=land4_height_absolute-land4, xmin=hxmin, xmax=hxmax, colors='darkred', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land4_height_absolute-land4, "4nd land", color='darkred', fontsize=6)

if land5 > 0:
  land5_height_absolute = skirt_length+groove5+land5
  ax1.hlines(y=land5_height_absolute, xmin=hxmin, xmax=hxmax, colors='gray', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land5_height_absolute, "5nd land", color='darkred', fontsize=6)
  ax1.hlines(y=land5_height_absolute-land5, xmin=hxmin, xmax=hxmax, colors='gray', linestyles='--',linewidth=hlinewidth, label='b')
  ax1.text(hxmax+2, land5_height_absolute-land5, "5nd land", color='darkred', fontsize=6)

# Skirt
if skirt_length > 0:
  skirt_length_absolute = skirt_length
  ax1.hlines(y=skirt_length_absolute, xmin=hxmin, xmax=hxmax, colors='blue', linestyles='--',linewidth=hlinewidth, label='e')
  ax1.text(hxmax+2, skirt_length_absolute, "Skirt", color='blue', fontsize=6)
  ax1.hlines(y=0, xmin=hxmin, xmax=hxmax, colors='blue', linestyles='--',linewidth=hlinewidth+1, label='f')
  ax1.text(hxmax+2, 0, "Skirt", color='blue', fontsize=6)

# Ausgabe der Formkurven-Abschnitte
hxmin= nom_diameter/2 + 12
hxmax=nom_diameter + 50
section = 1
for ll in level_limit:
  ax1.hlines(y=ll, xmin=hxmin, xmax=hxmax, colors='black', linestyles='-',linewidth=hlinewidth)
  if (section % 2) > 0:
    ax1.text(nom_diameter / 2 + 20, ll +1, "Section " + str(section), color='black', fontsize=6)
  section = section +1

# Formkurve plotten
xa, ya, xe, ye = readGeometrieProfileCurves(root)

# Plot 2 erstellen
#plt.figure(figsize=(10, 8))  # Set the figure size (width, height) in inches
# plt.ylim(0, int(total_length*2))

xpl = []
ypl = []
i = 0
# Reihenfolge umdrehen
xa = xa[::-1]
ya = ya[::-1]
xe = xe[::-1]
ye = ye[::-1]
for x,y,x2,y2 in zip(xa,ya,xe,ye):
    xpl.append(x)
    xpl.append(x2)
    ypl.append(y)
    ypl.append(y2)
    i = i+1
# Ausgabebereich für Formkurve
allowed_y_range = [] #[0, 30.8, 36.27, 47.65]
# Ausgabe der Formkurven-Abschnitte
hxmin= nom_diameter / 2
hxmax=nom_diameter + 300
section = 1
for ll in level_limit:
  allowed_y_range.append(ll)
  ax2.hlines(y=ll, xmin=hxmin, xmax=hxmax, colors='black', linestyles='-',linewidth=hlinewidth)
  if (section % 2) > 0:
    ax2.text(nom_diameter / 2 + 20, ll +1, "Section " + str(section), color='black', fontsize=6)
  section = section +1


p1 = []
p2 =[]
p3 = []
p4 =[]
p5 = []
p6 =[]
p7 =[]
p8 =[]
p9 =[]
p10 =[]
allowed_y_len = len(allowed_y_range)
xmax = max(xpl)
plt.xlim(0, int(nom_diameter/2+xmax))
#print("xmax",xmax)
#print("allowed_y_range",allowed_y_range)
for x, y in zip(xpl, ypl):
    if (allowed_y_len >= 2 and y >= allowed_y_range[0]  and y <= allowed_y_range[1]):
        p1.append(x)
        p2.append(y)
    if (allowed_y_len >= 4 and y >= allowed_y_range[2]  and y <= allowed_y_range[3]):
        p3.append(x)
        p4.append(y)
    if (allowed_y_len >= 6 and y >= allowed_y_range[4]  and y <= allowed_y_range[5]):
        p5.append(x)
        p6.append(y)
    if (allowed_y_len >= 8 and y >= allowed_y_range[6]  and y <= allowed_y_range[7]):
        p7.append(x)
        p8.append(y)
    if (allowed_y_len >= 10 and y >= allowed_y_range[8]  and y <= allowed_y_range[9]):
        p9.append(x)
        p10.append(y)

print(p1)
print(p2)
ax2.plot(p1, p2, marker='.', color ='C0')
ax2.plot(p3, p4, marker='.', color ='C0')
ax2.plot(p5, p6, marker='.', color ='C0')
ax2.plot(p7, p8, marker='.', color ='C0')
ax2.plot(p9, p10, marker='.', color ='C0')

#plt.xlabel("X [mm]")
#plt.ylabel("Y [mm]")
ax1.set_xlabel("x [mm]")
ax2.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")
ax2.set_ylabel("y [mm]")
plt.title("profile curve")
ax1.grid(True)
ax2.grid(True)
# plt.legend()
plt.show()





In [ ]:
for ll in level_limit:
  print(ll)

skirt_y0 = 0
skirt_y1 = skirt_y0 + skirt_length
print(skirt_y0, skirt_y1)

land4_y0 = skirt_y1 + groove4
land4_y1 = land4_y0 + land4
print(land4_y0,land4_y1)

land3_y0 = land4_y1 + groove3
land3_y1 = land3_y0 + land3
print(land3_y0,land3_y1)

land2_y0 = land3_y1 +groove2
land2_y1 = land2_y0 + land2
print(land2_y0,land2_y1)

topland_y0 = land2_y1 +groove1
topland_y1 = topland_y0 + top_land
print(topland_y0,topland_y1)


In [ ]:

def find_range_within_level_limit(skirt_y0, skirt_y1, level_limit):
  """
  Findet den Bereich, in dem skirt_y0 und skirt_y1 innerhalb der level_limit liegen.

  Args:
    skirt_y0: Der Startwert von skirt.
    skirt_y1: Der Endwert von skirt.
    level_limit: Eine Liste von level Limits.

  Returns:
    Eine Liste von level_limit, in der skirt_y0 und skirt_y1 liegen.
  """
  range_within_limit = []
  for i in range(len(level_limit) - 1):
    if skirt_y0 >= level_limit[i] and skirt_y1 <= level_limit[i+1]:
      range_within_limit.append(level_limit[i])
      range_within_limit.append(level_limit[i+1])
      break
  return range_within_limit

# Beispiel:
'''
level_limit = [0, 30.8, 36.27, 47.65]
skirt_y0 = 31
skirt_y1 = 34
'''
level_limit, level_limit_number = readProfileLimits(root)
i =0
range_found = find_range_within_level_limit(skirt_y0, skirt_y1, level_limit)
for a in range_found:
  if i == 0:
    print("skirt im bereich von:", a)
  else:
    print("skirt im bereich bis:", a)
    i = 0
  i = i+1


i =0
range_found = find_range_within_level_limit(land4_y0, land4_y1, level_limit)
for a in range_found:
  if i == 0:
    print("4land im bereich von:", a)
  else:
    print("4land im bereich bis:", a)
    i = 0
  i = i+1


i =0
range_found = find_range_within_level_limit(land3_y0, land3_y1, level_limit)
for a in range_found:
  if i == 0:
    print("3land im bereich von:", a)
  else:
    print("3land im bereich bis:", a)
    i = 0
  i = i+1

i =0
range_found = find_range_within_level_limit(land2_y0, land2_y1, level_limit)
for a in range_found:
  if i == 0:
    print("2land im bereich von:", a)
  else:
    print("2land im bereich bis:", a)
    i = 0
  i = i+1

i =0
a1=0
a2=0
range_found = find_range_within_level_limit(topland_y0, topland_y1, level_limit)
for a in range_found:
  if i == 0:
    a1 = a
  else:
    a2
    i = 0
  i = i+1
if i == 0:
  print("topland nicht definiert im bereich ", topland_y0, topland_y1)
else:
  print("topland definiert im bereich ", topland_y0, topland_y1)

In [ ]:
# prompt: range_found die werte einzeln an index auslesen

def find_range_within_level_limit(skirt_y0, skirt_y1, level_limit):
  """
  Findet den Bereich, in dem skirt_y0 und skirt_y1 innerhalb der level_limit liegen.

  Args:
    skirt_y0: Der Startwert von skirt.
    skirt_y1: Der Endwert von skirt.
    level_limit: Eine Liste von level Limits.

  Returns:
    Eine Liste von level_limit, in der skirt_y0 und skirt_y1 liegen.
  """
  range_within_limit = []
  for i in range(len(level_limit) - 1):
    if skirt_y0 >= level_limit[i] and skirt_y1 <= level_limit[i+1]:
      range_within_limit.append(level_limit[i])
      range_within_limit.append(level_limit[i+1])
      break
  return range_within_limit

# Beispiel:
# level_limit = [0, 30.8, 36.27, 47.65]
# skirt_y0 = 31
# skirt_y1 = 34

level_limit, level_limit_number = readProfileLimits(root)  # Annahme, dass diese Funktion definiert ist

range_found = find_range_within_level_limit(skirt_y0, skirt_y1, level_limit)

if range_found:
  for index, limit in enumerate(level_limit):
    if limit == range_found[0]:
      print("Index für den Startwert:", index)
      break

  for index, limit in enumerate(level_limit):
    if limit == range_found[1]:
      print("Index für den Endwert:", index)
      break
else:
  print("Kein Bereich innerhalb der Level Limits gefunden.")

